<a href="https://colab.research.google.com/github/mahdi-alalawi/AndroidTutorialForBeginners/blob/master/%D9%87%D9%8A%D8%AF%D8%AC_%D9%83%D9%88%D8%A8%D8%A7%D9%8A%D9%84%D8%AA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import time  # استيراد مكتبة الوقت

# بدء الاتصال مع MT5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

# تسجيل الدخول
account = 83967  # أدخل رقم حسابك
password = "NU6-caQp"  # أدخل كلمة المرور
server = "RoyalInvestment-Server"  # أدخل اسم السيرفر

if not mt5.login(account, password, server):
    print("login failed")
    mt5.shutdown()


def get_and_clean_gold_prices(symbol="XAUUSD", timeframe=mt5.TIMEFRAME_M15, num_bars=1000):
    """Reads gold prices from MetaTrader 5 for a given symbol, cleans the data, and returns it."""
    try:
        rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_bars)
        df = pd.DataFrame(rates)
        df['time'] = pd.to_datetime(df['time'], unit='s')
        df.set_index('time', inplace=True)

        required_columns = ['close', 'high', 'low', 'volume']
        for col in required_columns:
            if col not in df.columns:
                print(f"Warning: '{col}' column is missing from the data.")

        df = df[[col for col in required_columns if col in df.columns]]
        return df
    except Exception as e:
        print(f"An error occurred while fetching gold prices: {e}")
        return None


def calculate_atr(data, period=14):
    """Calculate Average True Range (ATR) for risk management."""
    high_low = data['high'] - data['low']
    high_close = abs(data['high'] - data['close'].shift(1))
    low_close = abs(data['low'] - data['close'].shift(1))
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    atr = tr.rolling(window=period).mean()
    return atr


def hedge_position_with_retrace_limit(symbol):
    """تطبيق استراتيجية الهيدج مع قيود على عدد الصفقات المفتوحة وحجم الصفقة الجديدة."""
    positions = mt5.positions_get(symbol=symbol)
    open_positions_count = len(positions)

    # تحقق من عدد الصفقات المفتوحة
    if open_positions_count >= 7:
        print("عدد الصفقات المفتوحة بلغ الحد الأقصى (7).")
        return

    if positions:
        for position in positions:
            # تحديد سعر الدخول ووقف الخسارة
            entry_price = position.price_open
            stop_loss = position.sl  # سعر وقف الخسارة

            # حساب المسافة بين سعر الدخول ووقف الخسارة
            distance_to_sl = entry_price - stop_loss if position.type == 0 else stop_loss - entry_price  # 0 = BUY, 1 = SELL

            # حساب السعر الذي يمثل 75% من المسافة نحو وقف الخسارة
            retrace_price = entry_price - (0.75 * distance_to_sl) if position.type == 0 else entry_price + (0.75 * distance_to_sl)  # 0 = BUY, 1 = SELL

            # الحصول على السعر الحالي
            current_price = mt5.symbol_info_tick(symbol).bid if position.type == 0 else mt5.symbol_info_tick(symbol).ask  # 0 = BUY, 1 = SELL

            # إذا كان السعر الحالي قد تراجع بنسبة 75% نحو وقف الخسارة
            if (position.type == 0 and current_price <= retrace_price) or \
               (position.type == 1 and current_price >= retrace_price):  # 0 = BUY, 1 = SELL

                # فتح مركز معكوس إذا كان العدد أقل من 7
                hedge_request = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "symbol": symbol,
                    "volume": position.volume,  # نفس حجم المركز الحالي
                    "type": 1 if position.type == 0 else 0,  # 1 = SELL, 0 = BUY
                    "price": mt5.symbol_info_tick(symbol).ask if position.type == 0 else mt5.symbol_info_tick(symbol).bid,
                    "sl": 0,
                    "tp": 0,
                    "magic": 123456,
                    "comment": "Hedge position",
                }

                # إرسال الطلب
                result = mt5.order_send(hedge_request)
                if result.retcode != mt5.TRADE_RETCODE_DONE:
                    print(f"فشل في فتح الصفقة: {result.retcode}")



def set_break_even(symbol):
    """تعيين وقف الخسارة إلى مستويات مختلفة بناءً على تقدم السعر نحو الهدف."""
    positions = mt5.positions_get(symbol=symbol)

    if positions:
        for position in positions:
            # الحصول على سعر الدخول وسعر جني الأرباح
            entry_price = position.price_open
            take_profit = position.tp  # سعر جني الأرباح

            # حساب المسافة بين سعر الدخول وسعر جني الأرباح
            distance_to_tp = take_profit - entry_price

            # الحصول على السعر الحالي
            current_price = mt5.symbol_info_tick(symbol).bid  # الحصول على السعر الحالي

            # حساب النسب المئوية
            fifty_percent_target = entry_price + (0.36 * distance_to_tp)
            seventy_five_percent_target = entry_price + (0.72 * distance_to_tp)
            ninety_percent_target = entry_price + (0.84 * distance_to_tp)

            # تعيين وقف الخسارة بناءً على تقدم السعر
            if current_price >= fifty_percent_target:
                new_stop_loss = entry_price + (0.12 * distance_to_tp)
                update_stop_loss(position, new_stop_loss)

            if current_price >= seventy_five_percent_target:
                new_stop_loss = entry_price + (0.24 * distance_to_tp)
                update_stop_loss(position, new_stop_loss)

            if current_price >= ninety_percent_target:
                new_stop_loss = entry_price + (0.48 * distance_to_tp)
                update_stop_loss(position, new_stop_loss)


def update_stop_loss(position, new_stop_loss):
    """تحديث وقف الخسارة لمركز معين."""
    request = {
        "action": mt5.TRADE_ACTION_SLTP,
        "position": position.ticket,
        "sl": new_stop_loss,  # تعيين وقف الخسارة الجديد
        "tp": position.tp,  # الحفاظ على جني الأرباح الحالي
        "magic": 123456,  # رقم سحري فريد
        "comment": "Update stop loss",
        "type": position.type,  # نوع الصفقة (شراء أو بيع)
    }
    result = mt5.order_send(request)
    if result.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Failed to update stop loss: {result.retcode}")


def calculate_lot_size(account_balance, risk_percentage=0.01, pip_value=10):
    """حساب حجم العقد بناءً على رصيد الحساب ونسبة المخاطرة."""
    risk_amount = account_balance * risk_percentage
    lot_size = risk_amount / pip_value  # يمكنك تعديل pip_value حسب الأداة المالية
    return max(0.01, lot_size)  # تأكد من أن حجم العقد لا يقل عن 0.01

def place_order(symbol, order_type, volume, risk_percentage=0.01):
    """Place an order in MetaTrader 5 with dynamic lot size."""
    try:
        account_info = mt5.account_info()
        if account_info is None:
            print("Failed to retrieve account info.")
            return

        account_balance = account_info.balance  # الحصول على رصيد الحساب
        # لا حاجة لحساب الحجم هنا، لأنه تم تمرير الحجم كمعامل

        if not mt5.symbol_select(symbol, True):
            print(f"Symbol {symbol} not found.")
            return

        price = mt5.symbol_info_tick(symbol).ask if order_type == "buy" else mt5.symbol_info_tick(symbol).bid
        order_type_mt5 = 0 if order_type == "buy" else 1  # 0 للشراء و 1 للبيع

        # حساب ATR لتحديد SL و TP
        atr = calculate_atr(get_and_clean_gold_prices(symbol))
        atr_value = atr.iloc[-1] if atr is not None else 0

        sl = price - 2 * atr_value if order_type == "buy" else price + 2 * atr_value
        tp = price + 3 * atr_value if order_type == "buy" else price - 3 * atr_value

        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": volume,
            "type": order_type_mt5,
            "price": price,
            "sl": sl,
            "tp": tp,
            "deviation": 10,
            "magic": 234000,
            "comment": "Python script order",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }

        result = mt5.order_send(request)

        if result is None:
            print("Order send failed. Result is None.")
            print(f"Error: {mt5.last_error()}")
            return

        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"Order failed: {result.retcode}, Error: {mt5.last_error()}")
        else:
            print(f"Order placed successfully: {order_type} {volume} lots of {symbol} at {price} with SL: {sl} and TP: {tp}")

    except Exception as e:
        print(f"An error occurred while placing an order: {e}")

def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    try:
        url = "https://www.investing.com/news/commodities-news/gold"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        news_headers = soup.select('div.news-item-header')
        news_titles = [header.text.strip() for header in news_headers]
        return news_titles
    except Exception as e:
        print(f"An error occurred while fetching news: {e}")
        return []


def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    try:
        for title in news_titles:
            analysis = TextBlob(title)
            score = analysis.sentiment.polarity
            sentiments.append(score)
    except Exception as e:
        print(f"An error occurred during sentiment analysis: {e}")
    return sentiments


def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    try:
        for i in range(1, len(data)):
            if data['close'].iloc[i] > data['close'].iloc[i - 1] and data['close'].iloc[i - 1] < data['close'].iloc[
                i - 2]:
                bullish_blocks.append((data.index[i - 1], data['close'].iloc[i - 1]))
            elif data['close'].iloc[i] < data['close'].iloc[i - 1] and data['close'].iloc[i - 1] > data['close'].iloc[
                i - 2]:
                bearish_blocks.append((data.index[i - 1], data['close'].iloc[i - 1]))
    except Exception as e:
        print(f"An error occurred while calculating order blocks: {e}")
    return bullish_blocks, bearish_blocks


def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['close'] = pd.to_numeric(data['close'], errors='coerce')
    data['low'] = pd.to_numeric(data['low'], errors='coerce')
    data['high'] = pd.to_numeric(data['high'], errors='coerce')
    data.dropna(subset=['close', 'low', 'high'], inplace=True)

    premium = data['close'] - data['low'].rolling(window=20).min()
    discount = data['high'].rolling(window=20).max() - data['close']
    return premium, discount


def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    try:
        for i in range(1, len(data)):
            if data['close'].iloc[i] > data['high'].iloc[i - 1]:
                supply_zones.append((data.index[i], data['high'].iloc[i - 1]))
            elif data['close'].iloc[i] < data['low'].iloc[i - 1]:
                demand_zones.append((data.index[i], data['low'].iloc[i - 1]))
    except Exception as e:
        print(f"An error occurred while calculating supply and demand zones: {e}")
    return supply_zones, demand_zones


def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    try:
        support = data['low'].rolling(window=20).min()
        resistance = data['high'].rolling(window=20).max()
        return support, resistance
    except Exception as e:
        print(f"An error occurred while calculating support and resistance: {e}")
        return None, None


def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }


def calculate_rsi(data, period=14):
    """حساب RSI"""
    try:
        delta = data['close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        data['RSI'] = 100 - (100 / (1 + rs))
        print(f"RSI: {data['RSI'].iloc[-1]}")
        return data
    except Exception as e:
        print(f"An error occurred while calculating RSI: {e}")
        return data


def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    try:
        data['MA'] = data['close'].rolling(window=period).mean()
        print(f"Moving Average (MA): {data['MA'].iloc[-1]}")
        return data
    except Exception as e:
        print(f"An error occurred while calculating moving average: {e}")
        return data


def calculate_momentum(data, period=10):
    """حساب الزخم"""
    try:
        data['Momentum'] = data['close'].diff(periods=period)
        print(f"Momentum: {data['Momentum'].iloc[-1]}")
        return data
    except Exception as e:
        print(f"An error occurred while calculating momentum: {e}")
        return data


def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    try:
        if 'volume' in data.columns:
            data['Volume_MA'] = data['volume'].rolling(window=20).mean()
            print(f"Volume Moving Average: {data['Volume_MA'].iloc[-1]}")
        return data
    except Exception as e:
        print(f"An error occurred while analyzing volume: {e}")
        return data


def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    try:
        for i in range(1, len(data) - 4):
            if (data['close'].iloc[i] < data['close'].iloc[i - 1] and
                    data['close'].iloc[i + 1] < data['close'].iloc[i] and
                    data['close'].iloc[i + 2] > data['close'].iloc[i + 1] and
                    data['close'].iloc[i + 3] < data['close'].iloc[i + 2] and
                    data['close'].iloc[i + 4] > data['close'].iloc[i + 3]):
                wave_points.append(data.index[i])
    except Exception as e:
        print(f"An error occurred while calculating wolf wave: {e}")
    return wave_points


def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    try:
        data['Price_Change'] = data['close'].pct_change()
        data['Log_Returns'] = np.log(data['close'] / data['close'].shift(1))
        data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
        return data
    except Exception as e:
        print(f"An error occurred while creating features: {e}")
        return data


def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    try:
        data['Channel_Upper'] = data['high'].rolling(window=window).max()
        data['Channel_Lower'] = data['low'].rolling(window=window).min()
        data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
        return data
    except Exception as e:
        print(f"An error occurred while calculating price channel: {e}")
        return data


def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    try:
        atr = calculate_atr(data)
        data['ATR'] = atr
        data['Stop_Loss'] = data['close'] - (data['ATR'] * 2)
        data['Take_Profit'] = data['close'] + (data['ATR'] * 3)
        print(
            f"ATR: {data['ATR'].iloc[-1]}, Stop Loss: {data['Stop_Loss'].iloc[-1]}, Take Profit: {data['Take_Profit'].iloc[-1]}")
        return data
    except Exception as e:
        print(f"An error occurred while managing risk: {e}")
        return data


def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    try:
        for i in range(2, len(البيانات)):
            if البيانات['close'].iloc[i] < البيانات['close'].iloc[i - 1] and البيانات['close'].iloc[i - 1] > \
                    البيانات['close'].iloc[i - 2]:
                مناطق_طلبات_الشراء.append((البيانات.index[i - 1], البيانات['close'].iloc[i - 1]))
            elif البيانات['close'].iloc[i] > البيانات['close'].iloc[i - 1] and البيانات['close'].iloc[i - 1] < \
                    البيانات['close'].iloc[i - 2]:
                مناطق_طلبات_البيع.append((البيانات.index[i - 1], البيانات['close'].iloc[i - 1]))

    except Exception as e:
        print(f"An error occurred while calculating order blocks: {e}")


def build_strategy(data):
    """بناء الاستراتيجية الكاملة مع تحليل جميع المؤشرات."""
    try:
        # إعداد البيانات
        data['target'] = data['close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
        data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

        # حساب جميع المؤشرات
        data = calculate_rsi(data)  # حساب مؤشر القوة النسبية
        data = calculate_moving_average(data)  # حساب المتوسط المتحرك
        data = calculate_momentum(data)  # حساب الزخم
        premium, discount = calculate_premium_discount(data)  # حساب العلاوة والخصم
        data['Premium'] = premium
        data['Discount'] = discount




        # تحديد الاتجاه العام باستخدام المتوسط المتحرك
        current_ma = data['MA'].iloc[-1]
        previous_ma = data['MA'].iloc[-2]

        # تحقق مما إذا كان الاتجاه قد تغير
        if current_ma > previous_ma:
            new_trend = "bullish"
        else:
            new_trend = "bearish"

        # تحقق من الصفقات المفتوحة
        open_positions = mt5.positions_get(symbol="XAUUSD")
        current_positions = "buy" if open_positions and open_positions[0].type == 0 else "sell"

        # إذا كان الاتجاه قد تغير، أغلق الصفقات الحالية وافتح صفقات جديدة
        if (new_trend == "bullish" and current_positions == "sell") or (new_trend == "bearish" and current_positions == "buy"):
            # إغلاق الصفقات المفتوحة
            for position in open_positions:
                request = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "position": position.ticket,
                    "type": 1 if position.type == 0 else 0,  # عكس نوع الصفقة
                    "volume": position.volume,
                    "price": mt5.symbol_info_tick("XAUUSD").bid if position.type == 0 else mt5.symbol_info_tick(
                        "XAUUSD").ask,
                    "deviation": 10,
                    "magic": 234000,
                    "comment": "Closing position..."
                }
                result = mt5.order_send(request)

                if result is None:
                    print("Order send failed. Result is None.")
                    print(f"Error: {mt5.last_error()}")
                    return

                if hasattr(result, 'retcode') and result.retcode != mt5.TRADE_RETCODE_DONE:
                    print(f"Order failed: {result.retcode}, Error: {mt5.last_error()}")
                else:
                    print(f"Closed position successfully: {position.ticket}")

            # فتح صفقة جديدة بناءً على الاتجاه الجديد
            order_type = "buy" if new_trend == "bullish" else "sell"
            volume = calculate_lot_size(account_balance=mt5.account_info().balance)
            print(f"Calculated volume: {volume}")

            price = mt5.symbol_info_tick("XAUUSD").ask if order_type == "buy" else mt5.symbol_info_tick("XAUUSD").bid

            # تعيين SL و TP
            sl = price - 10 if order_type == "buy" else price + 10
            tp = price + 20 if order_type == "buy" else price - 20

            request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": "XAUUSD",
                "volume": volume,
                "type": 0 if order_type == "buy" else 1,  # 0 للشراء و 1 للبيع
                "price": price,
                "sl": sl,
                "tp": tp,
                "deviation": 10,
                "magic": 234000,
                "comment": "Opening new position..."
            }

            result = mt5.order_send(request)

            if result is None:
                print("Order send failed. Result is None.")
                print(f"Error: {mt5.last_error()}")
                return

            if hasattr(result, 'retcode') and result.retcode != mt5.TRADE_RETCODE_DONE:
                print(f"Order failed: {result.retcode}, Error: {mt5.last_error()}")
            else:
                print(f"Order placed successfully: {order_type} {volume} lots of XAUUSD at {price}")





        # حساب مناطق العرض والطلب
        supply_zones, demand_zones = calculate_supply_demand(data)
        support, resistance = calculate_support_resistance(data)
        data['Support'] = support
        data['Resistance'] = resistance

        # حساب مستويات فيبوناتشي
        fib_levels = fibonacci_retracement(data['high'].max(), data['low'].min())
        for key, value in fib_levels.items():
            data[key] = value

        # حساب نقاط وولف ويف
        wolf_wave_points = calculate_wolf_wave(data)

        # جلب الأخبار وتحليل المشاعر
        news_titles = fetch_news()
        sentiments = analyze_sentiment(news_titles)
        average_sentiment = np.mean(sentiments) if sentiments else 0
        print(f"Average Sentiment: {average_sentiment}")

        # استخدام تحليل المشاعر لتعديل استراتيجيتك
        if average_sentiment > 0.1 and not check_open_positions("XAUUSD"):
            place_order("XAUUSD", "buy", volume=0.01)
        elif average_sentiment < -0.1 and not check_open_positions("XAUUSD"):
            place_order("XAUUSD", "sell", volume=0.01)

        # تأمين الصفقة إذا كانت هناك خسارة كبيرة
        if check_open_positions("XAUUSD"):
            hedge_position_with_retrace_limit("XAUUSD")  # تأمين الصفقة
            set_break_even("XAUUSD")  # تعيين بريك إيفن

            # إغلاق الصفقات الرابحة إذا كانت مجموع الأرباح تعادل ثلاث أرباع مجموع الخسائر
            positions = mt5.positions_get(symbol="XAUUSD")
            total_profit = sum(position.profit for position in positions if position.profit > 0)
            total_loss = sum(-position.profit for position in positions if position.profit < 0)

            # تحقق من الشرط
            if total_profit >= (3/4) * total_loss and total_loss > 0:  # تأكد من وجود خسائر
                for position in positions:
                    if position.profit > 0:  # إغلاق الصفقة الرابحة
                        request = {
                            "action": mt5.TRADE_ACTION_DEAL,
                            "position": position.ticket,
                            "type": 1 if position.type == 0 else 0,  # عكس نوع الصفقة
                            "volume": position.volume,
                            "price": mt5.symbol_info_tick("XAUUSD").bid if position.type == 0 else mt5.symbol_info_tick("XAUUSD").ask,
                            "deviation": 10,
                            "magic": 234000,
                            "comment": "Closing profitable position",
                            "type_time": mt5.ORDER_TIME_GTC,
                            "type_filling": mt5.ORDER_FILLING_IOC,
                        }
                        result = mt5.order_send(request)
                        if result.retcode == mt5.TRADE_RETCODE_DONE:
                            print(f"Closed profitable position for {position.ticket}.")

        # إنشاء ميزات إضافية
        data = create_features(data)  # إنشاء ميزات إضافية
        data = calculate_price_channel(data)  # حساب قناة السعر
        data = advanced_risk_management(data)  # إدارة المخاطر المتقدمة

        # حساب مناطق الأوامر
        bullish_blocks, bearish_blocks = calculate_order_blocks(data)

        # إعداد الميزات للمدخلات في نموذج التعلم الآلي
        features = data.drop(['target'], axis=1).fillna(0)  # الميزات
        target = data['target'].fillna(0)  # الهدف

        # تقسيم البيانات
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

        # تدريب النموذج
        model_xgb = XGBRegressor()
        model_xgb.fit(X_train, y_train)

        # التنبؤ
        predictions = model_xgb.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        print(f'Root Mean Squared Error: {rmse}')

        # تحديد أوامر الشراء والبيع بناءً على التنبؤ
        last_row = data.iloc[-1]
        predicted_price = model_xgb.predict(data.drop(['target'], axis=1).iloc[-1:].fillna(0))[0]

        print(f"Predicted Price: {predicted_price}, Current Price: {last_row['close']}")

        if not check_open_positions("XAUUSD"):  # تحقق من عدم وجود صفقات مفتوحة
            if predicted_price > last_row['close']:
                place_order("XAUUSD", "buy", volume=0.01)
            elif predicted_price < last_row['close']:
                place_order("XAUUSD", "sell", volume=0.01)

        # رسم مناطق الطلب والعرض
        احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")


def check_open_positions(symbol):
    """Check if there are any open positions for the given symbol."""
    positions = mt5.positions_get(symbol=symbol)
    return len(positions) > 0


# تنفيذ الاستراتيجية بشكل مستمر
if __name__ == "__main__":
    while True:
        data = get_and_clean_gold_prices()
        if data is not None:
            build_strategy(data)
        time.sleep(60)  # الانتظار لمدة دقيقة قبل التكرار


In [ ]:
# تعديل السطر في دالة build_strategy
volume = calculate_lot_size(account_balance=mt5.account_info().balance, risk_percentage=0.01)


In [ ]:
def smart_hedge(symbol, risk_percentage=0.01, retrace_percentage=0.75, break_even_percentage=0.5, profit_target_percentage=1.5):
    """
    Smart Hedging Function with Stop-Loss, Take-Profit, and Break-Even.

    Args:
        symbol (str): The trading symbol (e.g., "XAUUSD").
        risk_percentage (float): The percentage of account balance to risk per trade (default: 0.01).
        retrace_percentage (float): The percentage retracement from entry to trigger a hedge (default: 0.75).
        break_even_percentage (float): The percentage profit to move SL to break-even (default: 0.5).
        profit_target_percentage (float): The percentage profit target (default: 1.5).
    """
    positions = mt5.positions_get(symbol=symbol)

    # Check if there are already two open positions
    if len(positions) >= 2:
        print("Maximum of two open positions reached. Not hedging.")
        return

    if positions:
        for position in positions:
            entry_price = position.price_open
            stop_loss = position.sl
            take_profit = position.tp

            # Calculate retracement price
            distance_to_sl = abs(entry_price - stop_loss)
            retrace_price = (
                entry_price - (retrace_percentage * distance_to_sl) if position.type == 0
                else entry_price + (retrace_percentage * distance_to_sl)
            )

            # Get current price
            current_price = mt5.symbol_info_tick(symbol).bid if position.type == 0 else mt5.symbol_info_tick(symbol).ask

            # Check if retracement condition is met
            if (position.type == 0 and current_price <= retrace_price) or (position.type == 1 and current_price >= retrace_price):
                # Calculate hedge volume
                volume = calculate_lot_size(account_balance=mt5.account_info().balance, risk_percentage=risk_percentage)

                # Calculate stop-loss and take-profit for the hedge
                hedge_sl = entry_price + (profit_target_percentage * distance_to_sl) if position.type == 0 else entry_price - (profit_target_percentage * distance_to_sl)
                hedge_tp = entry_price - (profit_target_percentage * distance_to_sl) if position.type == 0 else entry_price + (profit_target_percentage * distance_to_sl)

                # Place the hedge order
                hedge_request = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "symbol": symbol,
                    "volume": volume,
                    "type": 1 if position.type == 0 else 0,  # Reverse order type
                    "price": current_price,
                    "sl": hedge_sl,
                    "tp": hedge_tp,
                    "magic": 123456,
                    "comment": "Hedge position",
                }

                result = mt5.order_send(hedge_request)

                if result.retcode != mt5.TRADE_RETCODE_DONE:
                    print(f"Failed to place hedge order: {result.retcode}")
                else:
                    print(f"Hedge order placed successfully. Ticket: {result.order}")

                    # Move original position's SL to break-even if break_even_percentage is reached
                    if (position.type == 0 and current_price >= entry_price + (break_even_percentage * distance_to_sl)) or \
                       (position.type == 1 and current_price <= entry_price - (break_even_percentage * distance_to_sl)):

                        break_even_sl = entry_price  # Set SL to entry price
                        update_stop_loss(position, break_even_sl)
                        print(f"Moved original position SL to break-even. Ticket: {position.ticket}")

    else:
        print("No open positions to hedge.")


In [ ]:
def smart_hedge(symbol, risk_percentage=0.01, retrace_percentage=0.75, break_even_percentage=0.5, profit_target_percentage=1.5):
    """
    دالة هيدج ذكية مع وقف الخسارة، وجني الربح، ونقطة التعادل.
    """
    positions = mt5.positions_get(symbol=symbol)

    # التحقق مما إذا كان هناك بالفعل مركزان مفتوحتان
    if len(positions) >= 2:
        print("تم الوصول إلى الحد الأقصى لعدد المراكز المفتوحة (2). لا يتم تنفيذ الهيدج.")
        return

    if positions:
        for position in positions:
            entry_price = position.price_open
            stop_loss = position.sl

            # حساب سعر التراجع
            distance_to_sl = abs(entry_price - stop_loss)
            retrace_price = entry_price - (retrace_percentage * distance_to_sl) if position.type == 0 else entry_price + (retrace_percentage * distance_to_sl)

            # الحصول على السعر الحالي
            current_price = mt5.symbol_info_tick(symbol).bid if position.type == 0 else mt5.symbol_info_tick(symbol).ask

            # التحقق مما إذا كان شرط التراجع قد تم استيفاؤه
            if (position.type == 0 and current_price <= retrace_price) or (position.type == 1 and current_price >= retrace_price):
                # حساب حجم الهيدج
                volume = calculate_lot_size(account_balance=mt5.account_info().balance, risk_percentage=risk_percentage)

                # حساب وقف الخسارة وجني الربح لمركز الهيدج
                hedge_sl = entry_price + (profit_target_percentage * distance_to_sl) if position.type == 0 else entry_price - (profit_target_percentage * distance_to_sl)
                hedge_tp = entry_price - (profit_target_percentage * distance_to_sl) if position.type == 0 else entry_price + (profit_target_percentage * distance_to_sl)

                # وضع أمر الهيدج
                hedge_request = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "symbol": symbol,
                    "volume": volume,
                    "type": 1 if position.type == 0 else 0,  # عكس نوع الأمر
                    "price": current_price,
                    "sl": hedge_sl,
                    "tp": hedge_tp,
                    "magic": 123456,
                    "comment": "مركز هيدج",
                }

                result = mt5.order_send(hedge_request)

                if result.retcode != mt5.TRADE_RETCODE_DONE:
                    print(f"فشل في وضع أمر الهيدج: {result.retcode}, الخطأ: {mt5.last_error()}")  # طباعة آخر خطأ للتصحيح
                else:
                    print(f"تم وضع أمر الهيدج بنجاح. رقم التذكرة: {result.order}")

                    # نقل وقف الخسارة للمركز الأصلي إلى نقطة التعادل إذا تم الوصول إلى break_even_percentage
                    if (position.type == 0 and current_price >= entry_price + (break_even_percentage * distance_to_sl)) or \
                       (position.type == 1 and current_price <= entry_price - (break_even_percentage * distance_to_sl)):

                        break_even_sl = entry_price  # تعيين وقف الخسارة إلى سعر الدخول
                        update_stop_loss(position, break_even_sl)
                        print(f"تم نقل وقف الخسارة للمركز الأصلي إلى نقطة التعادل. رقم التذكرة: {position.ticket}")